# Frame in Square
This project aims is to frame the provided images in a white square of provided size.

In [ ]:
import os
import glob
from PIL import Image

In [ ]:
INPUT_DIR_PATH = "/host/workdir/photos_size"
OUTPUT_DIR_PATH = "/host/workdir/bordure"
print(f"Working on host {os.environ['WORKDIR_PHOTOS']}")

In [ ]:
les_input_images = glob.glob(os.path.join(INPUT_DIR_PATH, "*/*.jpg"))
print(f"Found {len(les_input_images)} images.")
print("\n".join(
    les_input_images[0:len(les_input_images)//10] 
    + ["..."])
     )

In [ ]:
def generate_output_path(input_path):
    return os.path.join(OUTPUT_DIR_PATH, os.path.basename(input_path)) 

In [ ]:
def export_frame_in_square(input_image_path, square_size=2000):
    
    output_image_path = generate_output_path(input_image_path)
    if os.path.isfile(output_image_path):
        print(f"{output_image_path} already exists, passing")
        return 0
    # Load image
    image = Image.open(input_image_path)
    
    # Get size 
    width, height = image.size
    background_color = (255, 255, 255)
    new_image = None
    
    # If already square
    if width == height:
        new_image = image
        
    elif width > height:
        new_image = Image.new(image.mode, (width, width), background_color)
        new_image.paste(image, (0, (width - height) // 2))
        
    else:
        new_image = Image.new(image.mode, (height, height), background_color)
        new_image.paste(image, ((height - width) // 2, 0))
    
    if square_size is not None:
        new_image = new_image.resize((square_size, square_size))
    
    new_image.save(output_image_path, quality=100)

In [ ]:
for index, image_path in enumerate(les_input_images):
    print(f"{index+1}/{len(les_input_images)}: {image_path}" + " "*30, end="\r")
    export_frame_in_square(image_path)